## Part 1 Build on word-level text to generate a fixed-length vector for each sentence

In [28]:
import numpy as np 
np.random.seed(2019)

from numpy import genfromtxt

import random as r
r.seed(2019)

import sys
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score,accuracy_score
from sklearn.metrics import confusion_matrix

import torch
from torch import nn
import torch.nn.functional as F
from torch.autograd import Variable
torch.manual_seed(2019)
torch.cuda.manual_seed_all(2019)

import os
os.environ['PYTHONHASHSEED'] = str(2019)


from sklearn.model_selection import train_test_split
from util import d, here

import pandas as pd
from argparse import ArgumentParser

import random, tqdm, sys, math, gzip
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

torch.backends.cudnn.benchmark = False

torch.backends.cudnn.deterministic = True

import gc
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit,StratifiedKFold

from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC

# import fine_tuned
import inference
import time

import shutil

## Part 1 Built-in Functions

In [29]:
## show size of variables
def sizeof_fmt(num, suffix='B'):
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)

## random over sampling instances belong to minority class in case of imbalanced dataset
def resample(X,Y,random_state=2019):
    X_reshape = np.reshape(X, (X.shape[0], X.shape[1]*X.shape[2]))
    rus = RandomOverSampler(random_state=random_state)
    X_res, y_res = rus.fit_resample(X_reshape, Y)
    X_back = np.reshape(X_res, (X_res.shape[0],X.shape[1],X.shape[2]))
    
    return X_back, y_res

## seed the initial data
def initial_seed_dataset(n_initial, Y,random_state):
    
    np.random.seed(random_state)
    
    df = pd.DataFrame()
    df['label'] = Y

    Samplesize = n_initial  #number of samples that you want       
    initial_samples = df.groupby('label', as_index=False).apply(lambda array: array.loc[np.random.choice(array.index, Samplesize, False),:])

    permutation = [index[1] for index in initial_samples.index.tolist()]
    
    print ('initial random chosen samples', permutation)
    
    return permutation

## compute the entropy
def compute_entropy(output):
    entropy = -(output[:,0]*np.log2(output[:,0]) + output[:,1]*np.log2(output[:,1]))
    return entropy


## sample the candidates for labelling
def sample_candidates(selection_method,permutation,y_prob,num_candidate,y_pred,dist):
    
    print('*'*20,'selection method ',selection_method,'*'*20)
    
        
        
    if selection_method == 'uncertainty':
       
        candidate_index = []
        
        for index in dist.argsort():
            if len(candidate_index) == num_candidate:
                break
            if index not in permutation:
                candidate_index.append(index)

        permutation = permutation+candidate_index
        
    if selection_method == 'certainty':
        entropy = compute_entropy(np.array(y_prob))
        candidate_index = []
        
        
        for index in entropy.argsort()[:]:
            if len(candidate_index) == num_candidate:
                break
            if index not in permutation:
                candidate_index.append(index)

        permutation = permutation+candidate_index
        
    if selection_method == 'mostConfident':
        entropy = compute_entropy(np.array(y_prob))
        candidate_index = []
        
        
        for index in np.argsort(np.array(y_prob)[:,1])[::-1]:
            if len(candidate_index) == num_candidate:
                break
            if index not in permutation:
                candidate_index.append(index)

        permutation = permutation+candidate_index
        
    print('*'*20,'num of training set ',len(permutation),'*'*20)   
        
    return permutation

## normalization instance
class Normalize(object):
    
    def normalize(self, X_train, X_val):
        self.scaler = MinMaxScaler()
        X_train = self.scaler.fit_transform(X_train)
        X_val   = self.scaler.transform(X_val)
       
        return (X_train, X_val) 
    
    def inverse(self, X_train, X_val):
        X_train = self.scaler.inverse_transform(X_train)
        X_val   = self.scaler.inverse_transform(X_val)
    
        return (X_train, X_val) 



    
## read data from directory
def read_data(path):

    corpora = []
    for filename in os.listdir(path):

        df_temp = pd.read_csv(path+filename)

        corpora.append(df_temp.text.tolist())

    class_one_len = len(corpora[0])
    class_two_len = len(corpora[1])

    return corpora, class_one_len, class_two_len

## construct training text dataset for fine tuning language model, save in directory data/
def construct_train_set(dataset,permutation):
    print('constructing new text training set.....')
    corpora, class_one_len, class_two_len = read_data('./corpus_data/'+dataset+'/')

    texts = np.array(corpora[0]+corpora[1])
    labels = np.array([0]*class_one_len+[1]*class_two_len)

#     X_train, X_test, y_train, y_test = train_test_split(texts[permutation], labels[permutation], test_size=0.0, random_state=2019)
    X_train, y_train = texts[permutation], labels[permutation]
    
    train_df = pd.DataFrame({'id':range(len(X_train)),'label':y_train,'alpha':['a']*len(X_train),'text':X_train})
#     dev_df = pd.DataFrame({'id':range(len(X_test)),'label':y_test,'alpha':['a']*len(X_test),'text':X_test})
    
    train_df.to_csv('data/train.tsv', sep='\t', index=False, header=False)
#     dev_df.to_csv('data/dev.tsv', sep='\t', index=False, header=False)

## Part 2. Import Inferred embeddings

In [32]:
## choose the bert-like models, options are roberta-base, disilbert-base-uncased, gpt2, xlnet-base-cased, albert-base-v2
text_rep = 'roberta-base'


## input dataset 
dataset = 'Longer_MovieReview'
dir_neg = dataset + '_neg.csv'
dir_pos = dataset + '_pos.csv'

representations_neg = genfromtxt('../active_datasets/%s_data/'%(text_rep)+dir_neg, delimiter=',')
representations_pos = genfromtxt('../acive_datasets/%s_data/'%(text_rep)+dir_pos, delimiter=',')

ulti_representations = np.concatenate((representations_neg,representations_pos),axis=0)
labels = np.array([0]*len(representations_neg)+[1]*len(representations_pos))

OSError: ../acive_datasets/roberta-base_data/Longer_MovieReview_pos.csv not found.

## Part 3. Active Learning Process

In [ ]:
NUM_CLS = 2

def active_process(arg):
    """
    Active learning procedure for normal active learning.
    """
    
    ## gridsearch for hyper-parameters tuning for SVM
    if arg.loop%arg.gridsearch_interval==0:
        gridsearch = True
    else:
        gridsearch = False
    
    if arg.initial: 
        ## the first round, initialize seed data
        permutation = initial_seed_dataset(arg.n_seedsamples,arg.LABEL_emb,arg.initial_random_seed)
       
        X_train = arg.TEXT_emb[permutation]
        Y_train = arg.LABEL_emb[permutation]

        X_val = arg.TEXT_emb
        Y_val = arg.LABEL_emb
        
        bin_count = np.bincount(Y_train)
        unique = np.unique(Y_train)
        print (
        'initial training set size:',
        Y_train.shape[0],
        'unique(labels):',
        unique,
        'label counts:',
        bin_count
        )

        print('Number of training examples ', len(Y_train))
        
    else:
        
        permutation = arg.permutation
       
        
        X_train = arg.TEXT_emb[permutation]
        Y_train = arg.LABEL_emb[permutation]

        X_val = arg.TEXT_emb
        Y_val = arg.LABEL_emb
        
        bin_count = np.bincount(Y_train)
        unique = np.unique(Y_train)
        print (
        'training set size:',
        Y_train.shape[0],
        'unique(labels):',
        unique,
        'label counts:',
        bin_count
        )

        print('Number of training examples ', len(Y_train))


    evaluation= { 'Confusion matrix':[]}
    
    normalizer = Normalize()
    X_train, X_val = normalizer.normalize(X_train, X_val) 
        
    
    if gridsearch == True:
       ## gridsearch for finding best hyper-parameters
        print('start gridsearch ...')
        parameters = [
                    {'kernel': ['linear'],
                     'C': [ 0.01, 0.1, 1,10]}]

        cv = StratifiedKFold(n_splits=5,random_state=arg.initial_random_seed)
        svc = SVC(probability=True,random_state=2019,class_weight='balanced',max_iter=10000)
        classifier = GridSearchCV(svc, parameters, cv=cv,scoring='accuracy',n_jobs=8,verbose = 0)
        classifier.fit(X_train, Y_train)
        print('best parameters is ', classifier.best_params_)
        best_params_ = classifier.best_params_

        
    else:
        ## hyper-parameters from last loop   
        best_params_ = arg.best_params_
        kernel = best_params_['kernel']
        C = best_params_['C']
        
        classifier = SVC(probability=True,random_state=2019,class_weight='balanced',C=C,kernel=kernel,max_iter=10000)
        classifier.fit(X_train, Y_train)
        print('best parameters is ', classifier)
        
        
    ## evaluation
        
    y_pred_prob = classifier.predict_proba(X_val)
#     y_pred = np.argmax(y_pred_prob, axis=1)
    y_pred = classifier.predict(X_val)
    y_true = Y_val
    
    numerators = classifier.decision_function(X_val)
    try:
        w_norm = np.linalg.norm(classifier.best_estimator_.coef_)
    except Exception:
        w_norm = np.linalg.norm(classifier.coef_)

    dist = abs(numerators) / w_norm
   

    y_true_remain = np.delete(np.array(y_true),permutation)
    y_pred_remain = np.delete(np.array(y_pred),permutation)
        
    print(classification_report(y_true_remain,y_pred_remain,labels=[0,1]))
   
    tn, fp, fn, tp = confusion_matrix(y_true_remain, y_pred_remain, labels=[0,1]).ravel()
    print('TP_H',bin_count[1],' TN_H',bin_count[0], ' TP_M',tp, ' TN_M',tn, ' FP_M', fp, ' FN_M',fn)
    acc = accuracy_score(y_true_remain, y_pred_remain)
    f_score = f1_score(y_true_remain,y_pred_remain,average='macro')
                
    evaluation['Confusion matrix'].append({'TP_H':bin_count[1],'TN_H':bin_count[0], 'TP_M':tp, 'TN_M':tn, 'FP_M': fp, 'FN_M':fn})

    
    return evaluation, y_pred_prob, y_pred,permutation, best_params_,dist

In [33]:
import easydict
from argparse import ArgumentParser

# Run the main function
# selection_method = 'certainty'
selection_method = 'uncertainty'
# selection_method = 'mostConfident'


if __name__ == "__main__":
    
    df = pd.DataFrame()
    df_permutation = pd.DataFrame()
    ### using different random seeds to seed the initial dataset
    random_seeds = [i for i in range(1988,1989)]
    
    start_time = time.time()
    for seed in random_seeds:
        confusion_matrice = []
        entropy = []
        permutations = []
        
        for loop in range(100):
            print('processing ',loop,'th loops---------------')

            if loop==0:
                
                
                parser = ArgumentParser()
                args = parser.parse_known_args()[0]
                args = easydict.EasyDict({
                        "n_seedsamples":5,
                        "initial_random_seed":seed,
                        "initial": True,
                        "TEXT_emb": ulti_representations,
                        "LABEL_emb": labels,
                        "gridsearch":True,
                        "loop":loop,
                        "gridsearch_interval":10

                })

                evaluation,y_pred_prob,y_pred,permutation,best_params_,dist = active_process(args)


                permutation = sample_candidates(num_candidate=10,permutation=permutation,selection_method=selection_method,y_prob=y_pred_prob,y_pred=y_pred,dist=dist)
                permutations.append(permutation)
                
                confusion_matrice.append(evaluation['Confusion matrix'])
                
                
        

            else:
                
#             

                parser = ArgumentParser()
                args = parser.parse_known_args()[0]
                args = easydict.EasyDict({
                        "n_seedsamples":5,
                        "initial_random_seed":seed,
                        "permutation": permutation,
                        "initial": False,
                        "TEXT_emb": ulti_representations,
                        "LABEL_emb": labels,
                        "best_params_":best_params_,
                        "loop":loop,
                        "gridsearch_interval":10
                })

                evaluation,y_pred_prob,y_pred,permutation,best_params_,dist = active_process(args)


                permutation = sample_candidates(num_candidate=10,permutation=permutation,selection_method=selection_method,y_prob=y_pred_prob,y_pred=y_pred,dist=dist)
                permutations.append(permutation)
                
                confusion_matrice.append(evaluation['Confusion matrix'])
                
               
                
              
            
        df[seed] = [item[0] for item in confusion_matrice]
        df.to_csv('./results/raw_%s_%s_%s_gridsearch_1.csv'%(text_rep,dataset,selection_method))


        df_permutation[seed] = [permutation for permutation in permutations]

        df_permutation.to_csv('./results/permutation_%s_%s_%s_result_1.csv'%(text_rep,dataset,selection_method))
        
        
    print("--- %s seconds ---" % (time.time() - start_time))

processing  0 th loops---------------
initial random chosen samples [476, 707, 770, 587, 468, 1026, 1891, 1549, 1101, 1390]
initial training set size: 10 unique(labels): [0 1] label counts: [5 5]
Number of training examples  10
start gridsearch ...


C:\Users\lujinghui\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


best parameters is  {'C': 0.01, 'kernel': 'linear'}
              precision    recall  f1-score   support

           0       0.70      0.83      0.76       995
           1       0.79      0.65      0.72       995

    accuracy                           0.74      1990
   macro avg       0.75      0.74      0.74      1990
weighted avg       0.75      0.74      0.74      1990

TP_H 5  TN_H 5  TP_M 650  TN_M 824  FP_M 171  FN_M 345
******************** selection method  uncertainty ********************
******************** num of training set  20 ********************
processing  1 th loops---------------
training set size: 20 unique(labels): [0 1] label counts: [11  9]
Number of training examples  20
best parameters is  SVC(C=0.01, break_ties=False, cache_size=200, class_weight='balanced',
    coef0=0.0, decision_function_shape='ovr', degree=3, gamma='scale',
    kernel='linear', max_iter=10000, probability=True, random_state=2019,
    shrinking=True, tol=0.001, verbose=False)
          

Number of training examples  100
best parameters is  SVC(C=0.01, break_ties=False, cache_size=200, class_weight='balanced',
    coef0=0.0, decision_function_shape='ovr', degree=3, gamma='scale',
    kernel='linear', max_iter=10000, probability=True, random_state=2019,
    shrinking=True, tol=0.001, verbose=False)
              precision    recall  f1-score   support

           0       0.79      0.95      0.87       973
           1       0.94      0.74      0.83       927

    accuracy                           0.85      1900
   macro avg       0.87      0.85      0.85      1900
weighted avg       0.86      0.85      0.85      1900

TP_H 73  TN_H 27  TP_M 683  TN_M 929  FP_M 44  FN_M 244
******************** selection method  uncertainty ********************
******************** num of training set  110 ********************
processing  10 th loops---------------
training set size: 110 unique(labels): [0 1] label counts: [31 79]
Number of training examples  110
start gridsearch ...


C:\Users\lujinghui\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


...

C:\Users\lujinghui\Anaconda3\lib\site-packages\sklearn\svm\_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


best parameters is  SVC(C=1, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=10000, probability=True, random_state=2019, shrinking=True,
    tol=0.001, verbose=False)
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       544
           1       0.99      0.99      0.99       556

    accuracy                           0.99      1100
   macro avg       0.99      0.99      0.99      1100
weighted avg       0.99      0.99      0.99      1100

TP_H 444  TN_H 456  TP_M 552  TN_M 537  FP_M 7  FN_M 4
******************** selection method  uncertainty ********************
******************** num of training set  910 ********************
processing  90 th loops---------------
training set size: 910 unique(labels): [0 1] label counts: [459 451]
Number of training examples  910
start gridsearch ...


C:\Users\lujinghui\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


best parameters is  {'C': 0.1, 'kernel': 'linear'}
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       541
           1       0.99      0.99      0.99       549

    accuracy                           0.99      1090
   macro avg       0.99      0.99      0.99      1090
weighted avg       0.99      0.99      0.99      1090

TP_H 451  TN_H 459  TP_M 545  TN_M 534  FP_M 7  FN_M 4
******************** selection method  uncertainty ********************
******************** num of training set  920 ********************
processing  91 th loops---------------
training set size: 920 unique(labels): [0 1] label counts: [463 457]
Number of training examples  920
best parameters is  SVC(C=0.1, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=10000, probability=True, random_state=2019, shrinking=True,
    tol=0.001, verbose=False)
      

training set size: 1000 unique(labels): [0 1] label counts: [499 501]
Number of training examples  1000
best parameters is  SVC(C=0.1, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=10000, probability=True, random_state=2019, shrinking=True,
    tol=0.001, verbose=False)
              precision    recall  f1-score   support

           0       1.00      0.99      0.99       501
           1       0.99      1.00      0.99       499

    accuracy                           0.99      1000
   macro avg       0.99      0.99      0.99      1000
weighted avg       0.99      0.99      0.99      1000

TP_H 501  TN_H 499  TP_M 497  TN_M 496  FP_M 5  FN_M 2
******************** selection method  uncertainty ********************
******************** num of training set  1010 ********************
--- 390.5867164134979 seconds ---


In [34]:
# import easydict
# from argparse import ArgumentParser

# # Run the main function
# # selection_method = 'certainty'
# selection_method = 'uncertainty'
# # selection_method = 'mostConfident'


# if __name__ == "__main__":
    
#     df = pd.DataFrame()
#     df_permutation = pd.DataFrame()
# #     random_seeds = [1989,1990,1991,1992,1993,1994,1995,1996,1997]
#     random_seeds = [i for i in range(1988,1998)]
    
#     start_time = time.time()
#     for seed in random_seeds:
#         confusion_matrice = []
#         entropy = []
#         permutations = []
        
#         for loop in range(100):
#             print('processing ',loop,'th loops---------------')

#             if loop==0 :
                
#                 dir_neg = dataset + '_neg.csv'
#                 dir_pos = dataset + '_pos.csv'
#                 representations_neg = genfromtxt('./%s_data/'%(text_rep)+dir_neg, delimiter=',')
#                 representations_pos = genfromtxt('./%s_data/'%(text_rep)+dir_pos, delimiter=',')
#                 ulti_representations = np.concatenate((representations_neg,representations_pos),axis=0)
#                 labels = np.array([0]*len(representations_neg)+[1]*len(representations_pos))
       
#                 parser = ArgumentParser()
#                 args = parser.parse_known_args()[0]
#                 args = easydict.EasyDict({
#                         "n_seedsamples":5,
#                         "initial_random_seed":seed,
#                         "initial": True,
#                         "TEXT_emb": ulti_representations,
#                         "LABEL_emb": labels,
#                         "gridsearch":True,
#                         "loop":loop,
#                         "gridsearch_interval":10

#                 })

#                 evaluation,y_pred_prob,y_pred,permutation,best_params_,dist = active_process(args)


#                 permutation = sample_candidates(num_candidate=10,permutation=permutation,selection_method=selection_method,y_prob=y_pred_prob,y_pred=y_pred,dist=dist)
#                 permutations.append(permutation)
                
#                 confusion_matrice.append(evaluation['Confusion matrix'])
                
                
        

#             else:
                
               
#                 if loop==40:
#                     construct_train_set(dataset,permutation)
#                     checkpoint, accs = fine_tuned.fine_tuned(loop,dataset,'roberta-base',32,1e-6)
#                     inference.inference(dataset,checkpoint,loop)

#                     dir_neg = dataset + '_tuned_neg_%s.csv'%(loop)
#                     dir_pos = dataset + '_tuned_pos_%s.csv'%(loop)
#                     representations_neg = genfromtxt('./%s_data/'%(text_rep)+dir_neg, delimiter=',')
#                     representations_pos = genfromtxt('./%s_data/'%(text_rep)+dir_pos, delimiter=',')

#                     ulti_representations = np.concatenate((representations_neg,representations_pos),axis=0)
#                 elif loop%40==0 and loop>40:
#                     construct_train_set(dataset,permutation)
#                     checkpoint, accs = fine_tuned.fine_tuned(loop,dataset,checkpoint,64,1e-5)
#                     inference.inference(dataset,checkpoint,loop)

#                     dir_neg = dataset + '_tuned_neg_%s.csv'%(loop)
#                     dir_pos = dataset + '_tuned_pos_%s.csv'%(loop)
#                     representations_neg = genfromtxt('./%s_data/'%(text_rep)+dir_neg, delimiter=',')
#                     representations_pos = genfromtxt('./%s_data/'%(text_rep)+dir_pos, delimiter=',')

#                     ulti_representations = np.concatenate((representations_neg,representations_pos),axis=0)
                    
              

#                 parser = ArgumentParser()
#                 args = parser.parse_known_args()[0]
#                 args = easydict.EasyDict({
#                         "n_seedsamples":5,
#                         "initial_random_seed":seed,
#                         "permutation": permutation,
#                         "initial": False,
#                         "TEXT_emb": ulti_representations,
#                         "LABEL_emb": labels,
#                         "best_params_":best_params_,
#                         "loop":loop,
#                         "gridsearch_interval":10
#                 })

#                 evaluation,y_pred_prob,y_pred,permutation,best_params_,dist = active_process(args)


#                 permutation = sample_candidates(num_candidate=10,permutation=permutation,selection_method=selection_method,y_prob=y_pred_prob,y_pred=y_pred,dist=dist)
#                 permutations.append(permutation)
                
#                 confusion_matrice.append(evaluation['Confusion matrix'])
                
#         shutil.rmtree("./outputs/")
                
                
            
#         df[seed] = [item[0] for item in confusion_matrice]
#         df.to_csv('./results/raw_%s_%s_%s_gridsearch.csv'%(text_rep,dataset,selection_method))


#         df_permutation[seed] = [permutation for permutation in permutations]

#         df_permutation.to_csv('./results/permutation_%s_%s_%s_result.csv'%(text_rep,dataset,selection_method))
        
#     print("--- %s seconds ---" % (time.time() - start_time))